# CPLEX IMPLEMENTATION

## Imports

In [724]:
import cplex
from cplex import Cplex
from cplex import SparsePair

import os
import pandas as pd

from collections import defaultdict

## Baby example

In [2]:
# Mini Example Create a problem instance
mini_problem = cplex.Cplex()
mini_problem.set_problem_type(cplex.Cplex.problem_type.LP)
mini_problem.objective.set_sense(mini_problem.objective.sense.minimize)

# Add variables
names = ["x1", "x2"]
obj = [3, 5]  # Coefficients in objective function
lb = [0, 0]   # Lower bounds
ub = [10, 10] # Upper bounds
mini_problem.variables.add(obj=obj, lb=lb, ub=ub, names=names)

# Add constraints
constraints = [
    [[0, 1], [1, 2]],  # 1*x1 + 2*x2 <= 10
    [[0, 1], [2, 1]]   # 2*x1 + 1*x2 <= 15
]
rhs = [10, 15]
senses = ["L", "L"]  # 'L' means ≤ constraint
mini_problem.linear_constraints.add(lin_expr=constraints, senses=senses, rhs=rhs)

# Solve
mini_problem.solve()
solution = mini_problem.solution.get_values()
print(f"Optimal values: {solution}")


Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 2 rows and 2 columns.
All rows and columns eliminated.
Presolve time = 0.02 sec. (0.00 ticks)
Optimal values: [0.0, 0.0]


## Define fixed sets: Time periods and road Widths

In [353]:
# Define fixed sets
T = tuple([1, 2, 3, 4, 5])  # 5 time periods
W = tuple(['timber','wide'])     # Road widths

## Load component-dependent sets

In [4]:
component_dir =r'1_Preprocessed_Data\3_Road_Network_Graphs\component_10'

### Load all nodes and create a mapping to name them shorter

In [182]:
file_path = f'{component_dir}\\4withsources_nodes.csv'
df = pd.read_csv(file_path)
df

,x,y,is_exit,is_source,stands
0,-15106.71098,150581.22331,False,False,"[945, 948]"
1,-16216.49208,150851.53241,True,False,[946]
2,-15464.13998,151201.64051,False,False,"[944, 1304, 947]"
3,-15761.71028,151228.89011,True,False,[944]
4,-15587.95728,151189.65731,False,False,"[944, 1304]"
5,-14863.04848,150875.44951,False,False,"[945, 942]"
6,-15443.44238,150603.44861,False,False,"[945, 947, 948]"
7,-15774.06238,150740.86021,False,False,"[944, 946, 947]"
8,-14861.44738,150934.48421,False,False,"[1304, 942]"
9,-15848.72048,150337.44601,False,False,"[947, 948]"


In [183]:
V = list(zip(df['x'], df['y'])) #csv has two columns x and y
V

[(-15106.71098, 150581.22331),
 (-16216.49208, 150851.53241),
 (-15464.13998, 151201.64051),
 (-15761.71028, 151228.89011),
 (-15587.95728, 151189.65731),
 (-14863.04848, 150875.44951),
 (-15443.44238, 150603.44861),
 (-15774.06238, 150740.86021),
 (-14861.44738, 150934.48421),
 (-15848.72048, 150337.44601),
 (-15387.94528, 151109.43011),
 (-15343.90788, 150903.59781),
 (-15969.22818, 150361.86671),
 (-15945.09588, 150839.36401),
 (-15069.81928, 150869.45261),
 (-15230.13938, 150665.38771),
 (-15369.24588, 150888.48011),
 (-15334.38388, 150628.08511),
 (-15842.48604, 151170.45481),
 (-16192.43465, 150699.23742),
 (-15289.32593, 150766.1793),
 (-15189.10959, 150864.98136),
 (-15172.98507, 151000.47245),
 (-15736.21395, 150994.90272),
 (-15057.87723, 150708.68595),
 (-16015.52817, 150676.42039),
 (-15581.13059, 150734.95184),
 (-15471.76829, 150383.40184),
 (-15370.22923, 151382.12787)]

In [184]:
# Create a dictionary mapping each node to a short name
node_mapping = {f'node{i + 1}': V[i] for i in range(len(V))}

# Print the mapping
print(node_mapping)

{'node1': (-15106.71098, 150581.22331), 'node2': (-16216.49208, 150851.53241), 'node3': (-15464.13998, 151201.64051), 'node4': (-15761.71028, 151228.89011), 'node5': (-15587.95728, 151189.65731), 'node6': (-14863.04848, 150875.44951), 'node7': (-15443.44238, 150603.44861), 'node8': (-15774.06238, 150740.86021), 'node9': (-14861.44738, 150934.48421), 'node10': (-15848.72048, 150337.44601), 'node11': (-15387.94528, 151109.43011), 'node12': (-15343.90788, 150903.59781), 'node13': (-15969.22818, 150361.86671), 'node14': (-15945.09588, 150839.36401), 'node15': (-15069.81928, 150869.45261), 'node16': (-15230.13938, 150665.38771), 'node17': (-15369.24588, 150888.48011), 'node18': (-15334.38388, 150628.08511), 'node19': (-15842.48604, 151170.45481), 'node20': (-16192.43465, 150699.23742), 'node21': (-15289.32593, 150766.1793), 'node22': (-15189.10959, 150864.98136), 'node23': (-15172.98507, 151000.47245), 'node24': (-15736.21395, 150994.90272), 'node25': (-15057.87723, 150708.68595), 'node26':

### Load Exit Nodes

In [185]:
# load exit nodes
file_path = f'{component_dir}\\exit_nodes.csv'
df = pd.read_csv(file_path)
V_exit = list(zip(df['x'], df['y'])) #csv has two columns x and y
V_exit

[(-16216.49208, 150851.53241),
 (-15761.71028, 151228.89011),
 (-15842.48604, 151170.45481),
 (-16192.43465, 150699.23742)]

In [186]:
V_exit_mapped = [short_name for node in V_exit for short_name, coords in node_mapping.items() if coords == node]
V_exit_mapped

['node2', 'node4', 'node19', 'node20']

### Load Source Nodes

In [187]:
# load source nodes
file_path = f'{component_dir}\\source_nodes.csv'
df = pd.read_csv(file_path)
V_source = { int(row['ID_UG']) : (row['x'], row['y']) for _, row in df.iterrows() }
V_source

{1469: (-15289.32593, 150766.1793),
 1470: (-15189.10959, 150864.98136),
 942: (-15172.98507, 151000.47245),
 944: (-15736.21395, 150994.90272),
 945: (-15057.87723, 150708.68595),
 946: (-16015.52817, 150676.42039),
 947: (-15581.13059, 150734.95184),
 948: (-15471.76829, 150383.40184),
 1304: (-15370.22923, 151382.12787)}

In [170]:
V_source_mapped = [short_name for node in V_source for short_name, coords in node_mapping.items() if coords == node]
V_source_mapped

[]

In [171]:
V_source.keys()

dict_keys([1469, 1470, 942, 944, 945, 946, 947, 948, 1304])

In [172]:
V_source_coord = set(V_source.values())
print(V_source_coord)
V_source_stands = set(V_source.keys())
print(V_source_stands)

{(-15289.32593, 150766.1793), (-15736.21395, 150994.90272), (-15471.76829, 150383.40184), (-15057.87723, 150708.68595), (-15370.22923, 151382.12787), (-15172.98507, 151000.47245), (-16015.52817, 150676.42039), (-15189.10959, 150864.98136), (-15581.13059, 150734.95184)}
{942, 944, 945, 946, 947, 948, 1304, 1469, 1470}


In [605]:
S = V_source_stands

### Load Arcs & Edges

In [659]:
# Load the arcs from the CSV files
arcs_fw_df = pd.read_csv(f'{component_dir}/arcsforward.csv')
arcs_bw_df = pd.read_csv(f'{component_dir}/arcsbackward.csv')
arcs_df = pd.read_csv(f'{component_dir}/arcs.csv')

# Assign the data to the respective variables
A = arcs_df.values.tolist()  # Convert the arcs data to a list of lists
A_fw = arcs_fw_df.values.tolist()  # List for forward arcs
A_bw = arcs_bw_df.values.tolist()  # List for backward arcs

In [660]:
# Convert arcs to sets of tuples for easy comparison
A_fw_set = {tuple(arc) for arc in A_fw}
A_bw_set = {tuple(arc) for arc in A_bw}

# Check for missing backward arcs
missing_bw = []
for arc in A_fw_set:
    reverse_arc = (arc[1], arc[0])
    if reverse_arc not in A_bw_set:
        missing_bw.append((arc, reverse_arc))

if missing_bw:
    print("⚠️ Some forward arcs have no reverse in A_bw:")
    for fw, expected_bw in missing_bw:
        print(f"Missing backward arc for forward arc {fw}: expected {expected_bw}")
else:
    print("✅ All forward arcs have corresponding backward arcs.")


✅ All forward arcs have corresponding backward arcs.


In [664]:
A[1]

['(-15106.71098, 150581.22331)', '(-14863.04848, 150875.44951)']

In [665]:
# Function to map coordinates to short names
def map_coordinates_to_short_name(coords):
    for short_name, node_coords in node_mapping.items():
        if tuple(map(float, coords.strip('()').split(', '))) == node_coords:
            return short_name
    return None  # Return None if the coordinates are not found

# Replace coordinates with their short names
A = [[map_coordinates_to_short_name(pair[0]), map_coordinates_to_short_name(pair[1])] for pair in A]
A_fw = [[map_coordinates_to_short_name(pair[0]), map_coordinates_to_short_name(pair[1])] for pair in A_fw]
A_bw = [[map_coordinates_to_short_name(pair[0]), map_coordinates_to_short_name(pair[1])] for pair in A_bw]
A[1]

['node1', 'node6']

In [677]:
A = tuple("".join(pair) for pair in A)
A_fw = tuple("".join(pair) for pair in A_fw)
A_bw = tuple("".join(pair) for pair in A_bw)
A_fw[1]

'node1node6'

In [688]:
# Create sets for fast lookup
A_fw_set = set(A_fw)
A_bw_set = set(A_bw)

# Helper function to swap halves
def swap_halves(arc):
    mid = len(arc) // 2
    return arc[mid:] + arc[:mid]

# Create the mapping from forward to backward (using half-swap)
fw_to_bw_map = {}
for arc in A_fw:
    swapped = swap_halves(arc)
    if swapped in A_bw_set:
        fw_to_bw_map[arc] = swapped

# Optional: backward to forward
bw_to_fw_map = {v: k for k, v in fw_to_bw_map.items()}


In [692]:
a = 'node1node6'  # Some arc from A_fw
reverse_a = fw_to_bw_map[a]
print(reverse_a)

node6node1


In [550]:
# Set E to be the same as arcs_forward (according to your instruction)
E = A_fw
print(len(A), len(A_fw), len(A_bw), len(E))
E

146 73 73 73


('node1node7',
 'node1node6',
 'node1node10',
 'node1node25',
 'node1node28',
 'node2node14',
 'node2node20',
 'node2node26',
 'node3node5',
 'node3node11',
 'node3node8',
 'node3node24',
 'node3node27',
 'node3node29',
 'node4node19',
 'node4node5',
 'node4node24',
 'node5node9',
 'node5node24',
 'node5node29',
 'node6node9',
 'node6node15',
 'node6node23',
 'node6node25',
 'node7node18',
 'node7node10',
 'node7node25',
 'node7node27',
 'node7node28',
 'node8node14',
 'node8node13',
 'node8node24',
 'node8node26',
 'node8node27',
 'node9node11',
 'node9node23',
 'node9node29',
 'node10node13',
 'node10node27',
 'node10node28',
 'node11node17',
 'node11node23',
 'node11node27',
 'node11node29',
 'node12node17',
 'node12node15',
 'node12node16',
 'node12node21',
 'node12node22',
 'node12node23',
 'node13node20',
 'node13node26',
 'node13node27',
 'node14node19',
 'node14node24',
 'node14node26',
 'node15node16',
 'node15node22',
 'node15node23',
 'node15node25',
 'node16node18',
 'node1

### Load Boundary nodes per stand

In [551]:
# load boundaries
boundaries_dict = {}

with open(f'{component_dir}\\boundaries.csv', 'r') as file:
    # Skip the header line
    next(file)
    for line in file:
        id_ug, nodes = line.strip().split(';')
        # Keep nodes as strings with parentheses intact
        boundaries_dict[id_ug] = [node.strip() for node in nodes.split(',')]

boundaries_dict

{'945': ['(-15106.71098',
  '150581.22331)',
  '(-14863.04848',
  '150875.44951)',
  '(-15443.44238',
  '150603.44861)',
  '(-15069.81928',
  '150869.45261)',
  '(-15230.13938',
  '150665.38771)',
  '(-15334.38388',
  '150628.08511)'],
 '948': ['(-15106.71098',
  '150581.22331)',
  '(-15443.44238',
  '150603.44861)',
  '(-15848.72048',
  '150337.44601)'],
 '946': ['(-16216.49208',
  '150851.53241)',
  '(-15774.06238',
  '150740.86021)',
  '(-15969.22818',
  '150361.86671)',
  '(-15945.09588',
  '150839.36401)',
  '(-16192.43465',
  '150699.23742)'],
 '944': ['(-15464.13998',
  '151201.64051)',
  '(-15761.71028',
  '151228.89011)',
  '(-15587.95728',
  '151189.65731)',
  '(-15774.06238',
  '150740.86021)',
  '(-15945.09588',
  '150839.36401)',
  '(-15842.48604',
  '151170.45481)'],
 '1304': ['(-15464.13998',
  '151201.64051)',
  '(-15587.95728',
  '151189.65731)',
  '(-14861.44738',
  '150934.48421)',
  '(-15387.94528',
  '151109.43011)'],
 '947': ['(-15464.13998',
  '151201.64051)',
  

In [178]:
# Reverse mapping: string of coords → short name
reverse_mapping = {
    f'({x[0]}, {x[1]})': short_name
    for short_name, x in node_mapping.items()
}

In [552]:
# Step 2: Recombine and map
mapped_boundaries_dict = {}

for ug_id, node_parts in boundaries_dict.items():
    # Rebuild coordinate strings from every 2 parts
    rebuilt_coords = [f'{node_parts[i].strip()}, {node_parts[i+1].strip()}' for i in range(0, len(node_parts), 2)]
    
    # Add parentheses back if needed
    rebuilt_coords = [f'({coord})' if not coord.startswith('(') else coord for coord in rebuilt_coords]
    
    # Map to short names using reverse_mapping
    mapped_nodes = [reverse_mapping.get(coord, None) for coord in rebuilt_coords]
    
    mapped_boundaries_dict[ug_id] = mapped_nodes
mapped_boundaries_dict    

{'945': ['node1', 'node6', 'node7', 'node15', 'node16', 'node18'],
 '948': ['node1', 'node7', 'node10'],
 '946': ['node2', 'node8', 'node13', 'node14', 'node20'],
 '944': ['node3', 'node4', 'node5', 'node8', 'node14', 'node19'],
 '1304': ['node3', 'node5', 'node9', 'node11'],
 '947': ['node3',
  'node7',
  'node8',
  'node10',
  'node11',
  'node13',
  'node17',
  'node18'],
 '942': ['node6', 'node9', 'node11', 'node12', 'node15', 'node17'],
 '1469': ['node12', 'node16', 'node17', 'node18'],
 '1470': ['node12', 'node15', 'node16']}

## Load parameters and actual values

### Load Costs

In [595]:
# load costs
edge_attr_df = pd.read_csv(f'{component_dir}/4withsources_edges_with_attributes.csv')
edge_attr_df

,"Node1(x,y)","Node2(x,y)",edgelength,has_exit,Build5m,Maintain5m,Build10m,Maintain10m,Upgrade,has_source,slope
0,"(-15106.71098, 150581.22331)","(-15443.44238, 150603.44861)",0.382454,False,1847,719,2772,719,1386,NaN,NaN
1,"(-15106.71098, 150581.22331)","(-14863.04848, 150875.44951)",0.511532,False,2471,960,3707,960,1854,NaN,NaN
2,"(-15106.71098, 150581.22331)","(-15848.72048, 150337.44601)",1.344332,False,6491,2520,9740,2520,4868,NaN,NaN
3,"(-15106.71098, 150581.22331)","(-15057.87723, 150708.68595)",0.000000,False,0,0,0,0,0,True,NaN
4,"(-15106.71098, 150581.22331)","(-15471.76829, 150383.40184)",0.000000,False,0,0,0,0,0,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...
68,"(-15334.38388, 150628.08511)","(-15289.32593, 150766.1793)",0.000000,False,0,0,0,0,0,True,NaN
69,"(-15334.38388, 150628.08511)","(-15057.87723, 150708.68595)",0.000000,False,0,0,0,0,0,True,NaN
70,"(-15334.38388, 150628.08511)","(-15581.13059, 150734.95184)",0.000000,False,0,0,0,0,0,True,NaN
71,"(-15842.48604, 151170.45481)","(-15736.21395, 150994.90272)",0.000000,False,0,0,0,0,0,True,NaN


In [596]:
# Map the Node1 and Node2 columns using the reverse mapping
edge_attr_df['1stNode_ID'] = edge_attr_df['Node1(x,y)'].map(reverse_mapping)
edge_attr_df['2ndNode_ID'] = edge_attr_df['Node2(x,y)'].map(reverse_mapping)
edge_attr_df

,"Node1(x,y)","Node2(x,y)",edgelength,has_exit,Build5m,Maintain5m,Build10m,Maintain10m,Upgrade,has_source,slope,1stNode_ID,2ndNode_ID
0,"(-15106.71098, 150581.22331)","(-15443.44238, 150603.44861)",0.382454,False,1847,719,2772,719,1386,NaN,NaN,node1,node7
1,"(-15106.71098, 150581.22331)","(-14863.04848, 150875.44951)",0.511532,False,2471,960,3707,960,1854,NaN,NaN,node1,node6
2,"(-15106.71098, 150581.22331)","(-15848.72048, 150337.44601)",1.344332,False,6491,2520,9740,2520,4868,NaN,NaN,node1,node10
3,"(-15106.71098, 150581.22331)","(-15057.87723, 150708.68595)",0.000000,False,0,0,0,0,0,True,NaN,node1,node25
4,"(-15106.71098, 150581.22331)","(-15471.76829, 150383.40184)",0.000000,False,0,0,0,0,0,True,NaN,node1,node28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,"(-15334.38388, 150628.08511)","(-15289.32593, 150766.1793)",0.000000,False,0,0,0,0,0,True,NaN,node18,node21
69,"(-15334.38388, 150628.08511)","(-15057.87723, 150708.68595)",0.000000,False,0,0,0,0,0,True,NaN,node18,node25
70,"(-15334.38388, 150628.08511)","(-15581.13059, 150734.95184)",0.000000,False,0,0,0,0,0,True,NaN,node18,node27
71,"(-15842.48604, 151170.45481)","(-15736.21395, 150994.90272)",0.000000,False,0,0,0,0,0,True,NaN,node19,node24


In [597]:
# Make sure both columns are strings first
edge_attr_df['EdgeID'] = edge_attr_df['1stNode_ID'].astype(str) + edge_attr_df['2ndNode_ID'].astype(str)
edge_attr_df

,"Node1(x,y)","Node2(x,y)",edgelength,has_exit,Build5m,Maintain5m,Build10m,Maintain10m,Upgrade,has_source,slope,1stNode_ID,2ndNode_ID,EdgeID
0,"(-15106.71098, 150581.22331)","(-15443.44238, 150603.44861)",0.382454,False,1847,719,2772,719,1386,NaN,NaN,node1,node7,node1node7
1,"(-15106.71098, 150581.22331)","(-14863.04848, 150875.44951)",0.511532,False,2471,960,3707,960,1854,NaN,NaN,node1,node6,node1node6
2,"(-15106.71098, 150581.22331)","(-15848.72048, 150337.44601)",1.344332,False,6491,2520,9740,2520,4868,NaN,NaN,node1,node10,node1node10
3,"(-15106.71098, 150581.22331)","(-15057.87723, 150708.68595)",0.000000,False,0,0,0,0,0,True,NaN,node1,node25,node1node25
4,"(-15106.71098, 150581.22331)","(-15471.76829, 150383.40184)",0.000000,False,0,0,0,0,0,True,NaN,node1,node28,node1node28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,"(-15334.38388, 150628.08511)","(-15289.32593, 150766.1793)",0.000000,False,0,0,0,0,0,True,NaN,node18,node21,node18node21
69,"(-15334.38388, 150628.08511)","(-15057.87723, 150708.68595)",0.000000,False,0,0,0,0,0,True,NaN,node18,node25,node18node25
70,"(-15334.38388, 150628.08511)","(-15581.13059, 150734.95184)",0.000000,False,0,0,0,0,0,True,NaN,node18,node27,node18node27
71,"(-15842.48604, 151170.45481)","(-15736.21395, 150994.90272)",0.000000,False,0,0,0,0,0,True,NaN,node19,node24,node19node24


### Set costs parameters

In [588]:
# Initialize cost dictionaries without t
CostC = {(e, w): None for e in E for w in W}
CostM = {(e, w): None for e in E for w in W}
CostU = {e: None for e in E}

# Iterate over the DataFrame rows to assign actual cost values (rounded to 2 decimals)
for _, row in edge_attr_df.iterrows():
    e = (row['EdgeID'])  # Create edge tuple
    
    CostC[(e, W[0])] = round(row['Build5m'])
    CostC[(e, W[1])] = round(row['Build10m'])
    CostM[(e, W[0])] = round(row['Maintain5m'])
    CostM[(e, W[1])] = round(row['Maintain10m'])
    CostU[e] = round(row['Upgrade'])
CostC 

{('node1node7', 'timber'): 1847,
 ('node1node7', 'wide'): 2772,
 ('node1node6', 'timber'): 2471,
 ('node1node6', 'wide'): 3707,
 ('node1node10', 'timber'): 6491,
 ('node1node10', 'wide'): 9740,
 ('node1node25', 'timber'): 0,
 ('node1node25', 'wide'): 0,
 ('node1node28', 'timber'): 0,
 ('node1node28', 'wide'): 0,
 ('node2node14', 'timber'): 3041,
 ('node2node14', 'wide'): 4562,
 ('node2node20', 'timber'): 3120,
 ('node2node20', 'wide'): 4677,
 ('node2node26', 'timber'): 0,
 ('node2node26', 'wide'): 0,
 ('node3node5', 'timber'): 615,
 ('node3node5', 'wide'): 922,
 ('node3node11', 'timber'): 619,
 ('node3node11', 'wide'): 929,
 ('node3node8', 'timber'): 2980,
 ('node3node8', 'wide'): 4472,
 ('node3node24', 'timber'): 0,
 ('node3node24', 'wide'): 0,
 ('node3node27', 'timber'): 0,
 ('node3node27', 'wide'): 0,
 ('node3node29', 'timber'): 0,
 ('node3node29', 'wide'): 0,
 ('node4node19', 'timber'): 1447,
 ('node4node19', 'wide'): 2171,
 ('node4node5', 'timber'): 880,
 ('node4node5', 'wide'): 1

### Load Access Needs

In [613]:
# load fire resistance and timber matrix
mrespath = r'1_Preprocessed_Data\0_Access_Requirements_Matrices\mres_stands_access_matrix.csv'
mwoodpath = r'1_Preprocessed_Data\0_Access_Requirements_Matrices\mwood_stands_access_matrix.csv'
stakepath = r'1_Preprocessed_Data\0_Access_Requirements_Matrices\stake_stands_access_matrix.csv'"gd prep outtakes.ipynb"

accessneeds_df = pd.read_csv(f'{mrespath}')
accessneeds_df.set_index('ID_UG', inplace=True)
accessneeds_df

,5mt1,5mt2,5mt3,5mt4,5mt5,10mt1,10mt2,10mt3,10mt4,10mt5
ID_UG,,,,,,,,,,
840,1,0,1,1,1,0,0,0,0,0
841,1,1,0,1,1,0,0,0,0,0
846,1,0,0,1,1,1,0,0,0,0
1204,1,0,0,1,1,0,0,0,0,0
1206,1,1,1,1,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1613,1,0,0,1,1,0,0,0,0,0
1614,1,0,0,1,1,0,0,0,0,0
1615,1,0,0,1,1,0,0,0,0,0


In [614]:
# filter for the current component
accessneeds_df = accessneeds_df[accessneeds_df.index.isin(S)]
accessneeds_df

,5mt1,5mt2,5mt3,5mt4,5mt5,10mt1,10mt2,10mt3,10mt4,10mt5
ID_UG,,,,,,,,,,
1469,1,0,1,1,1,0,0,0,0,0
1470,1,0,1,1,1,0,0,0,0,0
942,1,0,1,1,1,0,0,0,0,0
945,1,0,0,1,1,0,0,0,0,0
947,1,1,0,1,1,0,0,0,0,0


In [615]:
# Modify column names to have t in T as column indices
accessneeds_df.columns = accessneeds_df.columns.str.split('t').str[-1].astype(int)
# Split the dataframe into two parts, one for 5m one for 10m
wide_accessneeds_df = accessneeds_df.iloc[:, -5:]  # Last 5 columns
accessneeds_df = accessneeds_df.iloc[:, :5]  # First 5 columns

print(accessneeds_df.head())
print(wide_accessneeds_df.head())

       1  2  3  4  5
ID_UG               
1469   1  0  1  1  1
1470   1  0  1  1  1
942    1  0  1  1  1
945    1  0  0  1  1
947    1  1  0  1  1
       1  2  3  4  5
ID_UG               
1469   0  0  0  0  0
1470   0  0  0  0  0
942    0  0  0  0  0
945    0  0  0  0  0
947    0  0  0  0  0


In [616]:
# doublecheck
T == accessneeds_df.columns

array([ True,  True,  True,  True,  True])

In [624]:
S

{942, 944, 945, 946, 947, 948, 1304, 1469, 1470}

In [621]:
accessneeds_df.index

Index([1469, 1470, 942, 945, 947], dtype='int64', name='ID_UG')

We only consider the inaccessible stands!

In [ ]:
S_inaccessible = set()
S_accessible = set()

for s in S: 
    if s in accessneeds_df.index: 
        S_inaccessible.add(s)
    else:
        S_accessible.add(s)  

print(S_inaccessible)
S = S_inaccessible

{942, 945, 947, 1469, 1470}


### Set needroad parameters
$$needroad_{s,t}^\text{timber}$$

$$needroad_{s,t}^\text{wide}$$

In [651]:
# First for 'timber'
needroad_timber = {
    ('timber', s, t): accessneeds_df.loc[s, t]
    for s in S
    for t in T
}

# Then for 'wide'
needroad_wide = {
    ('wide', s, t): wide_accessneeds_df.loc[s, t]
    for s in S
    for t in T
}

# Combine both dictionaries
needroad_w_s_t = {**needroad_timber, **needroad_wide}
needroad_w_s_t

{('timber', 942, 1): 1,
 ('timber', 942, 2): 0,
 ('timber', 942, 3): 1,
 ('timber', 942, 4): 1,
 ('timber', 942, 5): 1,
 ('timber', 945, 1): 1,
 ('timber', 945, 2): 0,
 ('timber', 945, 3): 0,
 ('timber', 945, 4): 1,
 ('timber', 945, 5): 1,
 ('timber', 947, 1): 1,
 ('timber', 947, 2): 1,
 ('timber', 947, 3): 0,
 ('timber', 947, 4): 1,
 ('timber', 947, 5): 1,
 ('timber', 1469, 1): 1,
 ('timber', 1469, 2): 0,
 ('timber', 1469, 3): 1,
 ('timber', 1469, 4): 1,
 ('timber', 1469, 5): 1,
 ('timber', 1470, 1): 1,
 ('timber', 1470, 2): 0,
 ('timber', 1470, 3): 1,
 ('timber', 1470, 4): 1,
 ('timber', 1470, 5): 1,
 ('wide', 942, 1): 0,
 ('wide', 942, 2): 0,
 ('wide', 942, 3): 0,
 ('wide', 942, 4): 0,
 ('wide', 942, 5): 0,
 ('wide', 945, 1): 0,
 ('wide', 945, 2): 0,
 ('wide', 945, 3): 0,
 ('wide', 945, 4): 0,
 ('wide', 945, 5): 0,
 ('wide', 947, 1): 0,
 ('wide', 947, 2): 0,
 ('wide', 947, 3): 0,
 ('wide', 947, 4): 0,
 ('wide', 947, 5): 0,
 ('wide', 1469, 1): 0,
 ('wide', 1469, 2): 0,
 ('wide', 1469

### Calculate maxflow parameters
$$maxflow_t^\text{timber} = \sum_{s\in S}needroad_{s,t}^\text{timber}$$

In [ ]:
w = 'timber'
maxflow_timber = {
    (w, t): sum(needroad_w_s_t[(w, s,t)] for s in S)
    for t in T
}
maxflow_timber


{('timber', 1): 5,
 ('timber', 2): 1,
 ('timber', 3): 3,
 ('timber', 4): 5,
 ('timber', 5): 5,
 ('wide', 1): 0,
 ('wide', 2): 0,
 ('wide', 3): 0,
 ('wide', 4): 0,
 ('wide', 5): 0}

$$maxflow_t^\text{wide} = \sum_{s\in S}needroad_{s,t}^\text{wide}$$

In [656]:
w = 'wide'
maxflow_wide = {
    (w, t): sum(needroad_w_s_t[(w, s,t)] for s in S)
    for t in T  # Loop over the time periods (columns)
}
maxflow_wide

{('wide', 1): 0,
 ('wide', 2): 0,
 ('wide', 3): 0,
 ('wide', 4): 0,
 ('wide', 5): 0}

In [657]:
# Combine both dictionaries
maxflow_w_t = {**maxflow_timber, **maxflow_wide}
maxflow_w_t

{('timber', 1): 5,
 ('timber', 2): 1,
 ('timber', 3): 3,
 ('timber', 4): 5,
 ('timber', 5): 5,
 ('wide', 1): 0,
 ('wide', 2): 0,
 ('wide', 3): 0,
 ('wide', 4): 0,
 ('wide', 5): 0}

## Build model

### Initialize model

In [739]:
# Initialize the CPLEX model
model = Cplex()
model.set_problem_type(Cplex.problem_type.LP)

### Decision variables

In [740]:
model.variables.delete()  # deletes all variables

#### Binary variables for construction maintenace upgrade

In [741]:
# Decision variables binary
C_vars = { (e, w, t): f"C_{e}_{w}_{t}" for e in E for w in W for t in (0,) + T }
M_vars = { (e, w, t): f"M_{e}_{w}_{t}" for e in E for w in W for t in (0,) + T }
U_vars = { (e, t): f"U_{e}_{t}" for e in E for t in (0,) + T }

In [742]:
len(E)*(len(T)+1)
print(len(C_vars), len(M_vars), len(U_vars))
C_vars

876 876 438


{('node1node7', 'timber', 0): 'C_node1node7_timber_0',
 ('node1node7', 'timber', 1): 'C_node1node7_timber_1',
 ('node1node7', 'timber', 2): 'C_node1node7_timber_2',
 ('node1node7', 'timber', 3): 'C_node1node7_timber_3',
 ('node1node7', 'timber', 4): 'C_node1node7_timber_4',
 ('node1node7', 'timber', 5): 'C_node1node7_timber_5',
 ('node1node7', 'wide', 0): 'C_node1node7_wide_0',
 ('node1node7', 'wide', 1): 'C_node1node7_wide_1',
 ('node1node7', 'wide', 2): 'C_node1node7_wide_2',
 ('node1node7', 'wide', 3): 'C_node1node7_wide_3',
 ('node1node7', 'wide', 4): 'C_node1node7_wide_4',
 ('node1node7', 'wide', 5): 'C_node1node7_wide_5',
 ('node1node6', 'timber', 0): 'C_node1node6_timber_0',
 ('node1node6', 'timber', 1): 'C_node1node6_timber_1',
 ('node1node6', 'timber', 2): 'C_node1node6_timber_2',
 ('node1node6', 'timber', 3): 'C_node1node6_timber_3',
 ('node1node6', 'timber', 4): 'C_node1node6_timber_4',
 ('node1node6', 'timber', 5): 'C_node1node6_timber_5',
 ('node1node6', 'wide', 0): 'C_nod

In [743]:
# Add C_vars
for (e, w, t), name in C_vars.items():
    if t == 0:
        model.variables.add(names=[name], lb=[0], ub=[0], types=["B"])  # fixed to 0
    else:
        model.variables.add(names=[name], lb=[0], ub=[1], types=["B"])

# Add M_vars
for (e, w, t), name in M_vars.items():
    if t == 0:
        model.variables.add(names=[name], lb=[0], ub=[0], types=["B"])
    else:
        model.variables.add(names=[name], lb=[0], ub=[1], types=["B"])

# Add U_vars
for (e, t), name in U_vars.items():
    if t == 0:
        model.variables.add(names=[name], lb=[0], ub=[0], types=["B"])
    else:
        model.variables.add(names=[name], lb=[0], ub=[1], types=["B"])

#### verify added decision variables
should be len(E) x (len (T)+1) x 5
because 5 different actions

In [744]:
n_action_variables = len(model.variables.get_names())
print(n_action_variables)
print(len(E)*(len(T)+1)*5)
model.variables.get_names()

2190
2190


['C_node1node7_timber_0',
 'C_node1node7_timber_1',
 'C_node1node7_timber_2',
 'C_node1node7_timber_3',
 'C_node1node7_timber_4',
 'C_node1node7_timber_5',
 'C_node1node7_wide_0',
 'C_node1node7_wide_1',
 'C_node1node7_wide_2',
 'C_node1node7_wide_3',
 'C_node1node7_wide_4',
 'C_node1node7_wide_5',
 'C_node1node6_timber_0',
 'C_node1node6_timber_1',
 'C_node1node6_timber_2',
 'C_node1node6_timber_3',
 'C_node1node6_timber_4',
 'C_node1node6_timber_5',
 'C_node1node6_wide_0',
 'C_node1node6_wide_1',
 'C_node1node6_wide_2',
 'C_node1node6_wide_3',
 'C_node1node6_wide_4',
 'C_node1node6_wide_5',
 'C_node1node10_timber_0',
 'C_node1node10_timber_1',
 'C_node1node10_timber_2',
 'C_node1node10_timber_3',
 'C_node1node10_timber_4',
 'C_node1node10_timber_5',
 'C_node1node10_wide_0',
 'C_node1node10_wide_1',
 'C_node1node10_wide_2',
 'C_node1node10_wide_3',
 'C_node1node10_wide_4',
 'C_node1node10_wide_5',
 'C_node1node25_timber_0',
 'C_node1node25_timber_1',
 'C_node1node25_timber_2',
 'C_nod

#### Integer variables for flow
should be 2 x len(E) x len(T) variables

In [745]:
# Decision variables integer
flow= { (a, w, t): f"flow_{w}_{a}_{t}" for a in A for w in W for t in T }
flow

{('node1node7', 'timber', 1): 'flow_timber_node1node7_1',
 ('node1node7', 'timber', 2): 'flow_timber_node1node7_2',
 ('node1node7', 'timber', 3): 'flow_timber_node1node7_3',
 ('node1node7', 'timber', 4): 'flow_timber_node1node7_4',
 ('node1node7', 'timber', 5): 'flow_timber_node1node7_5',
 ('node1node7', 'wide', 1): 'flow_wide_node1node7_1',
 ('node1node7', 'wide', 2): 'flow_wide_node1node7_2',
 ('node1node7', 'wide', 3): 'flow_wide_node1node7_3',
 ('node1node7', 'wide', 4): 'flow_wide_node1node7_4',
 ('node1node7', 'wide', 5): 'flow_wide_node1node7_5',
 ('node1node6', 'timber', 1): 'flow_timber_node1node6_1',
 ('node1node6', 'timber', 2): 'flow_timber_node1node6_2',
 ('node1node6', 'timber', 3): 'flow_timber_node1node6_3',
 ('node1node6', 'timber', 4): 'flow_timber_node1node6_4',
 ('node1node6', 'timber', 5): 'flow_timber_node1node6_5',
 ('node1node6', 'wide', 1): 'flow_wide_node1node6_1',
 ('node1node6', 'wide', 2): 'flow_wide_node1node6_2',
 ('node1node6', 'wide', 3): 'flow_wide_nod

In [746]:
# Add variables to the model
for key, name in flow.items():
    model.variables.add(names=[name], lb=[0], ub=[212], types=["I"])

In [747]:
# verify 
n_flow_variables = len(model.variables.get_names()) - n_action_variables
print(n_flow_variables)
print(len(A)*len(T)*len(W))
model.variables.get_names()[-730:]

1460
1460


['flow_timber_node24node14_1',
 'flow_timber_node24node14_2',
 'flow_timber_node24node14_3',
 'flow_timber_node24node14_4',
 'flow_timber_node24node14_5',
 'flow_wide_node24node14_1',
 'flow_wide_node24node14_2',
 'flow_wide_node24node14_3',
 'flow_wide_node24node14_4',
 'flow_wide_node24node14_5',
 'flow_timber_node24node19_1',
 'flow_timber_node24node19_2',
 'flow_timber_node24node19_3',
 'flow_timber_node24node19_4',
 'flow_timber_node24node19_5',
 'flow_wide_node24node19_1',
 'flow_wide_node24node19_2',
 'flow_wide_node24node19_3',
 'flow_wide_node24node19_4',
 'flow_wide_node24node19_5',
 'flow_timber_node27node3_1',
 'flow_timber_node27node3_2',
 'flow_timber_node27node3_3',
 'flow_timber_node27node3_4',
 'flow_timber_node27node3_5',
 'flow_wide_node27node3_1',
 'flow_wide_node27node3_2',
 'flow_wide_node27node3_3',
 'flow_wide_node27node3_4',
 'flow_wide_node27node3_5',
 'flow_timber_node27node7_1',
 'flow_timber_node27node7_2',
 'flow_timber_node27node7_3',
 'flow_timber_node27

### Objective Function

In [748]:
# Objective function
objective_terms = []
for t in T:
    for w in W:
        for e in E:  # Iterate over edges
            # Add terms for C and M decision variables
            
            # C_vars[(e, w, t)] retrieves the decision variable associated with edge `e`, weight `w`, and time `t`
            # CostC[(e, w)] gives the build cost associated with edge `e` and weight `w`
            objective_terms.append((C_vars[(e, w, t)], CostC[(e, w)]))
            objective_terms.append((M_vars[(e, w, t)], CostM[(e, w)]))
            objective_terms.append((U_vars[(e, t)], CostU[e]))
print(objective_terms)            

[('C_node1node7_timber_1', 1847), ('M_node1node7_timber_1', 719), ('U_node1node7_1', 1386), ('C_node1node6_timber_1', 2471), ('M_node1node6_timber_1', 960), ('U_node1node6_1', 1854), ('C_node1node10_timber_1', 6491), ('M_node1node10_timber_1', 2520), ('U_node1node10_1', 4868), ('C_node1node25_timber_1', 0), ('M_node1node25_timber_1', 0), ('U_node1node25_1', 0), ('C_node1node28_timber_1', 0), ('M_node1node28_timber_1', 0), ('U_node1node28_1', 0), ('C_node2node14_timber_1', 3041), ('M_node2node14_timber_1', 1087), ('U_node2node14_1', 2281), ('C_node2node20_timber_1', 3120), ('M_node2node20_timber_1', 1113), ('U_node2node20_1', 2339), ('C_node2node26_timber_1', 0), ('M_node2node26_timber_1', 0), ('U_node2node26_1', 0), ('C_node3node5_timber_1', 615), ('M_node3node5_timber_1', 239), ('U_node3node5_1', 461), ('C_node3node11_timber_1', 619), ('M_node3node11_timber_1', 241), ('U_node3node11_1', 464), ('C_node3node8_timber_1', 2980), ('M_node3node8_timber_1', 1162), ('U_node3node8_1', 2235), (

In [749]:
# Create a dictionary to accumulate coefficients for each variable to avoid adding duplicate variables to CPLEX model
accumulated_terms = defaultdict(float)

# Loop over each time period (T), weight (W), and edge (E)
for t in T:  
    for w in W:
        for e in E:  
            # Add coefficients for C, M, and U decision variables
            accumulated_terms[C_vars[(e, w, t)]] += CostC[(e, w)]
            accumulated_terms[M_vars[(e, w, t)]] += CostM[(e, w)]
            accumulated_terms[U_vars[(e, t)]] += CostU[e]

# Now create the objective_terms list from accumulated_terms
objective_terms = [(var, coef) for var, coef in accumulated_terms.items()]
print(objective_terms)

[('C_node1node7_timber_1', 1847.0), ('M_node1node7_timber_1', 719.0), ('U_node1node7_1', 2772.0), ('C_node1node6_timber_1', 2471.0), ('M_node1node6_timber_1', 960.0), ('U_node1node6_1', 3708.0), ('C_node1node10_timber_1', 6491.0), ('M_node1node10_timber_1', 2520.0), ('U_node1node10_1', 9736.0), ('C_node1node25_timber_1', 0.0), ('M_node1node25_timber_1', 0.0), ('U_node1node25_1', 0.0), ('C_node1node28_timber_1', 0.0), ('M_node1node28_timber_1', 0.0), ('U_node1node28_1', 0.0), ('C_node2node14_timber_1', 3041.0), ('M_node2node14_timber_1', 1087.0), ('U_node2node14_1', 4562.0), ('C_node2node20_timber_1', 3120.0), ('M_node2node20_timber_1', 1113.0), ('U_node2node20_1', 4678.0), ('C_node2node26_timber_1', 0.0), ('M_node2node26_timber_1', 0.0), ('U_node2node26_1', 0.0), ('C_node3node5_timber_1', 615.0), ('M_node3node5_timber_1', 239.0), ('U_node3node5_1', 922.0), ('C_node3node11_timber_1', 619.0), ('M_node3node11_timber_1', 241.0), ('U_node3node11_1', 928.0), ('C_node3node8_timber_1', 2980.0)

In [750]:
# Set the objective function
model.objective.set_linear(objective_terms)
model.objective.set_sense(model.objective.sense.minimize)

In [751]:
# verify
model.objective.get_linear()

[0.0,
 1847.0,
 1847.0,
 1847.0,
 1847.0,
 1847.0,
 0.0,
 2772.0,
 2772.0,
 2772.0,
 2772.0,
 2772.0,
 0.0,
 2471.0,
 2471.0,
 2471.0,
 2471.0,
 2471.0,
 0.0,
 3707.0,
 3707.0,
 3707.0,
 3707.0,
 3707.0,
 0.0,
 6491.0,
 6491.0,
 6491.0,
 6491.0,
 6491.0,
 0.0,
 9740.0,
 9740.0,
 9740.0,
 9740.0,
 9740.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 3041.0,
 3041.0,
 3041.0,
 3041.0,
 3041.0,
 0.0,
 4562.0,
 4562.0,
 4562.0,
 4562.0,
 4562.0,
 0.0,
 3120.0,
 3120.0,
 3120.0,
 3120.0,
 3120.0,
 0.0,
 4677.0,
 4677.0,
 4677.0,
 4677.0,
 4677.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 615.0,
 615.0,
 615.0,
 615.0,
 615.0,
 0.0,
 922.0,
 922.0,
 922.0,
 922.0,
 922.0,
 0.0,
 619.0,
 619.0,
 619.0,
 619.0,
 619.0,
 0.0,
 929.0,
 929.0,
 929.0,
 929.0,
 929.0,
 0.0,
 2980.0,
 2980.0,
 2980.0,
 2980.0,
 2980.0,
 0.0,
 4472.0,
 4472.0,
 4472.0,
 4472.

### Constraints

#### Initialize

In [861]:
# init
model.linear_constraints.delete()

#### Maintenance & Upgrade Constraints for timber roads
$\begin{align}
M_{e,t}^{\text{timber}} + U_{e,t} &\leq C_{e,t-1}^{\text{timber}} + M_{e,t-1}^{\text{timber}} \\
\\
&\Leftrightarrow \\
\\
M_{e,t}^{\text{timber}} + U_{e,t} - C_{e,t-1}^{\text{timber}} - M_{e,t-1}^{\text{timber}} &\leq 0
\end{align}$


In [ ]:
# Add Maintenance or Upgrade constraint for 5-meter roads
w = 'timber'
for t in T:
    for e in E:
        try:
            # Define the linear expression components for better clarity
            expr_vars = [M_vars[(e, w, t)], U_vars[(e, t)], C_vars[(e, w, t-1)], M_vars[(e, w, t-1)]]
            expr_coeffs = [1, 1, -1, -1]
            
            # Add the constraint to the model
            model.linear_constraints.add(
                lin_expr=[[expr_vars, expr_coeffs]],
                senses=["L"],  # 'L' indicates "less than or equal to"
                rhs=[0],  # Right-hand side of the constraint
                names=[f"Constraint_maintain_upgrade_{w}_{e}_{t}"]
            )
            
            # Print what was added to the model (variables and coefficients)
            print(f"Added constraint with name: maintain_upgrade_{w}_{e}_{t}")
            print(f"  Variables: {expr_vars}")
            print(f"  Coefficients: {expr_coeffs}")

        except KeyError as ke:
            print(f"KeyError: {ke} - This indicates a missing variable for (e={e}, t={t})")

Added constraint with name: maintain_upgrade_timber_node1node7_1
  Variables: ['M_node1node7_timber_1', 'U_node1node7_1', 'C_node1node7_timber_0', 'M_node1node7_timber_0']
  Coefficients: [1, 1, -1, -1]
Added constraint with name: maintain_upgrade_timber_node1node6_1
  Variables: ['M_node1node6_timber_1', 'U_node1node6_1', 'C_node1node6_timber_0', 'M_node1node6_timber_0']
  Coefficients: [1, 1, -1, -1]
Added constraint with name: maintain_upgrade_timber_node1node10_1
  Variables: ['M_node1node10_timber_1', 'U_node1node10_1', 'C_node1node10_timber_0', 'M_node1node10_timber_0']
  Coefficients: [1, 1, -1, -1]
Added constraint with name: maintain_upgrade_timber_node1node25_1
  Variables: ['M_node1node25_timber_1', 'U_node1node25_1', 'C_node1node25_timber_0', 'M_node1node25_timber_0']
  Coefficients: [1, 1, -1, -1]
Added constraint with name: maintain_upgrade_timber_node1node28_1
  Variables: ['M_node1node28_timber_1', 'U_node1node28_1', 'C_node1node28_timber_0', 'M_node1node28_timber_0']
 

##### verify

In [863]:
print(len(model.linear_constraints.get_names()))
n_c1 = len(model.linear_constraints.get_names())
for i in range(model.linear_constraints.get_num()):
    indices = model.linear_constraints.get_rows(i).ind
    coefs = model.linear_constraints.get_rows(i).val
    name = model.linear_constraints.get_names(i)
    rhs   = model.linear_constraints.get_rhs(i)
    sense = model.linear_constraints.get_senses(i)

    vars_in_constraint = [model.variables.get_names(idx) for idx in indices]
    terms = [f"{coef}*{var}" for coef, var in zip(coefs, vars_in_constraint)]
    expression = " + ".join(terms)
    
    print(f"{name or 'unnamed'}: {expression} {sense} {rhs}")   

365
maintain_upgrade_timber_node1node7_1: -1.0*C_node1node7_timber_0 + -1.0*M_node1node7_timber_0 + 1.0*M_node1node7_timber_1 + 1.0*U_node1node7_1 L 0.0
maintain_upgrade_timber_node1node6_1: -1.0*C_node1node6_timber_0 + -1.0*M_node1node6_timber_0 + 1.0*M_node1node6_timber_1 + 1.0*U_node1node6_1 L 0.0
maintain_upgrade_timber_node1node10_1: -1.0*C_node1node10_timber_0 + -1.0*M_node1node10_timber_0 + 1.0*M_node1node10_timber_1 + 1.0*U_node1node10_1 L 0.0
maintain_upgrade_timber_node1node25_1: -1.0*C_node1node25_timber_0 + -1.0*M_node1node25_timber_0 + 1.0*M_node1node25_timber_1 + 1.0*U_node1node25_1 L 0.0
maintain_upgrade_timber_node1node28_1: -1.0*C_node1node28_timber_0 + -1.0*M_node1node28_timber_0 + 1.0*M_node1node28_timber_1 + 1.0*U_node1node28_1 L 0.0
maintain_upgrade_timber_node2node14_1: -1.0*C_node2node14_timber_0 + -1.0*M_node2node14_timber_0 + 1.0*M_node2node14_timber_1 + 1.0*U_node2node14_1 L 0.0
maintain_upgrade_timber_node2node20_1: -1.0*C_node2node20_timber_0 + -1.0*M_node2n

#### Maintenance and Upgrade constraints for wide roads
$\begin{align}
M_{e,t}^{wide} &\leq C_{e,t-1}^{wide} + M_{e,t-1}^{wide} + U_{e,t-1}\\
\\
&\Leftrightarrow\\\
\\
M_{e,t}^{wide} - C_{e,t-1}^{wide} - M_{e,t-1}^{wide} - U_{e,t-1} &\leq 0
\end{align}
$

In [ ]:
# Add Maintenance or Upgrade constraint for wide roads
w = 'wide'
for t in T:
    for e in E:
        try:
            # Define the linear expression components for better clarity
            expr_vars = [M_vars[(e, w, t)], C_vars[(e, w, t-1)], M_vars[(e, w, t-1)], U_vars[(e, t-1)]]
            expr_coeffs = [1, -1, -1, -1]
            print(expr_vars)
            print(expr_coeffs)
            
            # Add the constraint to the model
            model.linear_constraints.add(
                lin_expr=[[expr_vars, expr_coeffs]],
                senses=["L"],  # 'L' indicates "less than or equal to"
                rhs=[0],  # Right-hand side of the constraint
                names=[f"Constraint_maintain_upgrade_{w}_{e}_{t}"]
            )
            
            # Print what was added to the model (variables and coefficients)
            print(f"Added constraint with name: maintain_upgrade_{w}_{e}_{t}")
            print(f"  Variables: {expr_vars}")
            print(f"  Coefficients: {expr_coeffs}")

        except KeyError as ke:
            print(f"KeyError: {ke} - This indicates a missing variable for (e={e}, t={t})")

['M_node1node7_wide_1', 'C_node1node7_wide_0', 'M_node1node7_wide_0', 'U_node1node7_0']
[1, -1, -1, -1]
Added constraint with name: maintain_upgrade_wide_node1node7_1
  Variables: ['M_node1node7_wide_1', 'C_node1node7_wide_0', 'M_node1node7_wide_0', 'U_node1node7_0']
  Coefficients: [1, -1, -1, -1]
['M_node1node6_wide_1', 'C_node1node6_wide_0', 'M_node1node6_wide_0', 'U_node1node6_0']
[1, -1, -1, -1]
Added constraint with name: maintain_upgrade_wide_node1node6_1
  Variables: ['M_node1node6_wide_1', 'C_node1node6_wide_0', 'M_node1node6_wide_0', 'U_node1node6_0']
  Coefficients: [1, -1, -1, -1]
['M_node1node10_wide_1', 'C_node1node10_wide_0', 'M_node1node10_wide_0', 'U_node1node10_0']
[1, -1, -1, -1]
Added constraint with name: maintain_upgrade_wide_node1node10_1
  Variables: ['M_node1node10_wide_1', 'C_node1node10_wide_0', 'M_node1node10_wide_0', 'U_node1node10_0']
  Coefficients: [1, -1, -1, -1]
['M_node1node25_wide_1', 'C_node1node25_wide_0', 'M_node1node25_wide_0', 'U_node1node25_0']

##### verify

In [865]:
print(len(model.linear_constraints.get_names()))
n_c2 = len(model.linear_constraints.get_names()) - n_c1
print(n_c2)
for i in range(365, model.linear_constraints.get_num()):
    indices = model.linear_constraints.get_rows(i).ind
    coefs = model.linear_constraints.get_rows(i).val
    name = model.linear_constraints.get_names(i)
    rhs   = model.linear_constraints.get_rhs(i)
    sense = model.linear_constraints.get_senses(i)

    vars_in_constraint = [model.variables.get_names(idx) for idx in indices]
    terms = [f"{coef}*{var}" for coef, var in zip(coefs, vars_in_constraint)]
    expression = " + ".join(terms)
    
    print(f"{name or 'unnamed'}: {expression} {sense} {rhs}")

730
365
maintain_upgrade_wide_node1node7_1: -1.0*C_node1node7_wide_0 + -1.0*M_node1node7_wide_0 + 1.0*M_node1node7_wide_1 + -1.0*U_node1node7_0 L 0.0
maintain_upgrade_wide_node1node6_1: -1.0*C_node1node6_wide_0 + -1.0*M_node1node6_wide_0 + 1.0*M_node1node6_wide_1 + -1.0*U_node1node6_0 L 0.0
maintain_upgrade_wide_node1node10_1: -1.0*C_node1node10_wide_0 + -1.0*M_node1node10_wide_0 + 1.0*M_node1node10_wide_1 + -1.0*U_node1node10_0 L 0.0
maintain_upgrade_wide_node1node25_1: -1.0*C_node1node25_wide_0 + -1.0*M_node1node25_wide_0 + 1.0*M_node1node25_wide_1 + -1.0*U_node1node25_0 L 0.0
maintain_upgrade_wide_node1node28_1: -1.0*C_node1node28_wide_0 + -1.0*M_node1node28_wide_0 + 1.0*M_node1node28_wide_1 + -1.0*U_node1node28_0 L 0.0
maintain_upgrade_wide_node2node14_1: -1.0*C_node2node14_wide_0 + -1.0*M_node2node14_wide_0 + 1.0*M_node2node14_wide_1 + -1.0*U_node2node14_0 L 0.0
maintain_upgrade_wide_node2node20_1: -1.0*C_node2node20_wide_0 + -1.0*M_node2node20_wide_0 + 1.0*M_node2node20_wide_1 + 

#### Flow enforces road existence wide roads
$
\begin{align}
flow_{(u,v),t}^{\text{wide}} + flow_{(v,u),t}^{\text{wide}} & \leq maxflow_t^{\text{wide}} \big(C _{e,t}^{\text{wide}} + M_{e,t}^{\text{wide}} +  U_{e,t}\big)\\
\\
& \Leftrightarrow\\
\\
flow_{(u,v),t}^{\text{wide}} + flow_{(v,u),t}^{\text{wide}} - maxflow_t^{\text{wide}} \big(C _{e,t}^{\text{wide}} + M_{e,t}^{\text{wide}} +  U_{e,t}\big) & \leq 0\\
\\
& \Leftrightarrow\\
\\
flow_{(u,v),t}^{\text{wide}} + flow_{(v,u),t}^{\text{wide}} - maxflow_t^{\text{wide}}C _{e,t}^{\text{wide}} - maxflow_t^{\text{wide}}M_{e,t}^{\text{wide}} - maxflow_t^{\text{wide}}U_{e,t} & \leq 0\\
\\
\end{align}
$

In [849]:
for t in T:
    print(maxflow_w_t[(w,t)])

0
0
0
0
0


In [ ]:
# add wide flow enforces timber roads
w = 'wide'
for t in T:
    for a_fw in A_fw:
        a_bw = fw_to_bw_map[a_fw]
        e = a_fw
        try:
            # Define the linear expression components for better clarity
            expr_vars = [flow[(a_fw, w, t)], flow[(a_bw, w, t)], C_vars[(e, w, t)], M_vars[(e, w, t)], U_vars[(e, t)]]
            coeff = int(maxflow_w_t[(w,t)])
            expr_coeffs = [1, 1, -coeff , -coeff, -coeff]
            
            # Add the constraint to the model
            model.linear_constraints.add(
                lin_expr=[[expr_vars, expr_coeffs]],
                senses=["L"],  # 'L' indicates "less than or equal to"
                rhs=[0],  # Right-hand side of the constraint
                names=[f"Constraint_flow_enforcing_road_{w}_{e}_{t}"]
            )
            
            # Print what was added to the model (variables and coefficients)
            print(f"Added constraint flow_enforcing_road_{w}_{e}_{t}")
            print(f"  Variables: {expr_vars}")
            print(f"  Coefficients: {expr_coeffs}")

        except KeyError as ke:
            print(f"KeyError: {ke} - This indicates a missing variable for (e={e}, t={t})")

Added constraint flow_enforcing_road_wide_node1node7_1
  Variables: ['flow_wide_node1node7_1', 'flow_wide_node7node1_1', 'C_node1node7_wide_1', 'M_node1node7_wide_1', 'U_node1node7_1']
  Coefficients: [1, 1, 0, 0, 0]
Added constraint flow_enforcing_road_wide_node1node6_1
  Variables: ['flow_wide_node1node6_1', 'flow_wide_node6node1_1', 'C_node1node6_wide_1', 'M_node1node6_wide_1', 'U_node1node6_1']
  Coefficients: [1, 1, 0, 0, 0]
Added constraint flow_enforcing_road_wide_node1node10_1
  Variables: ['flow_wide_node1node10_1', 'flow_wide_node10node1_1', 'C_node1node10_wide_1', 'M_node1node10_wide_1', 'U_node1node10_1']
  Coefficients: [1, 1, 0, 0, 0]
Added constraint flow_enforcing_road_wide_node1node25_1
  Variables: ['flow_wide_node1node25_1', 'flow_wide_node25node1_1', 'C_node1node25_wide_1', 'M_node1node25_wide_1', 'U_node1node25_1']
  Coefficients: [1, 1, 0, 0, 0]
Added constraint flow_enforcing_road_wide_node1node28_1
  Variables: ['flow_wide_node1node28_1', 'flow_wide_node28node1_

##### verify
should be 5 x 73 constraints

In [ ]:
print(len(model.linear_constraints.get_names()))
n_c3 = len(model.linear_constraints.get_names()) - n_c1 - n_c2
print(n_c3)
for i in range(730, model.linear_constraints.get_num()):
    indices = model.linear_constraints.get_rows(i).ind
    coefs = model.linear_constraints.get_rows(i).val
    name = model.linear_constraints.get_names(i)
    rhs   = model.linear_constraints.get_rhs(i)
    sense = model.linear_constraints.get_senses(i)

    vars_in_constraint = [model.variables.get_names(idx) for idx in indices]
    terms = [f"{coef}*{var}" for coef, var in zip(coefs, vars_in_constraint)]
    expression = " + ".join(terms)
    
    print(f"{name or 'unnamed'}: {expression} {sense} {rhs}")

1095
365
flow_enforcing_road_wide_node1node7_1: 1.0*flow_wide_node1node7_1 + 1.0*flow_wide_node7node1_1 L 0.0
flow_enforcing_road_wide_node1node6_1: 1.0*flow_wide_node1node6_1 + 1.0*flow_wide_node6node1_1 L 0.0
flow_enforcing_road_wide_node1node10_1: 1.0*flow_wide_node1node10_1 + 1.0*flow_wide_node10node1_1 L 0.0
flow_enforcing_road_wide_node1node25_1: 1.0*flow_wide_node1node25_1 + 1.0*flow_wide_node25node1_1 L 0.0
flow_enforcing_road_wide_node1node28_1: 1.0*flow_wide_node1node28_1 + 1.0*flow_wide_node28node1_1 L 0.0
flow_enforcing_road_wide_node2node14_1: 1.0*flow_wide_node2node14_1 + 1.0*flow_wide_node14node2_1 L 0.0
flow_enforcing_road_wide_node2node20_1: 1.0*flow_wide_node2node20_1 + 1.0*flow_wide_node20node2_1 L 0.0
flow_enforcing_road_wide_node2node26_1: 1.0*flow_wide_node2node26_1 + 1.0*flow_wide_node26node2_1 L 0.0
flow_enforcing_road_wide_node3node5_1: 1.0*flow_wide_node3node5_1 + 1.0*flow_wide_node5node3_1 L 0.0
flow_enforcing_road_wide_node3node11_1: 1.0*flow_wide_node3node1

#### Flow (timber) enforcing road existence
$\begin{align}
flow_{(u,v),t}^{\text{timber}} + flow_{(v,u),t}^{\text{timber}}\leq maxflow_{t}^{\text{timber}} \big(C_{e,t}^{\text{timber}}+ M_{e,t}^{\text{timber}} + C _{e,t}^{\text{wide}} + M_{e,t}^{\text{wide}} +  U_{e,t}\big)\\
\\
&\Leftrightarrow
\\\\
flow_{(u,v),t}^{\text{timber}} + flow_{(v,u),t}^{\text{timber}}- maxflow_{t}^{\text{timber}} \big(C_{e,t}^{\text{timber}}+ M_{e,t}^{\text{timber}} + C _{e,t}^{\text{wide}} + M_{e,t}^{\text{wide}} +  U_{e,t}\big) &\leq 0\\
\\
&\Leftrightarrow
\\\\
flow_{(u,v),t}^{\text{timber}} + flow_{(v,u),t}^{\text{timber}}- maxflow_{t}^{\text{timber}} C_{e,t}^{\text{timber}}-maxflow_{t}^{\text{timber}}  M_{e,t}^{\text{timber}} -maxflow_{t}^{\text{timber}}  C _{e,t}^{\text{wide}} -maxflow_{t}^{\text{timber}}  M_{e,t}^{\text{wide}} -maxflow_{t}^{\text{timber}}  U_{e,t} &\leq 0\\
\end{align}$

In [871]:
# add timber flow enforces at least timber roads
for t in T:
    for a_fw in E:
        a_bw = fw_to_bw_map[a_fw]
        e = a_fw
        try:
            # Define the linear expression components for better clarity
            expr_vars = [flow[(a_fw, 'timber', t)], flow[(a_bw, 'timber', t)], C_vars[(e, 'timber', t)], M_vars[(e, 'timber', t)], C_vars[(e, 'wide', t)], M_vars[(e, 'wide', t)],U_vars[(e, t)]]
            coeff = int(maxflow_w_t[('timber',t)])
            expr_coeffs = [1, 1, -coeff , -coeff, -coeff, -coeff, -coeff]
            
            # Add the constraint to the model
            model.linear_constraints.add(
                lin_expr=[[expr_vars, expr_coeffs]],
                senses=["L"],  # 'L' indicates "less than or equal to"
                rhs=[0],  # Right-hand side of the constraint
                names=[f"Constraint_flow_enforcing_road_{w}_{e}_{t}"]
            )
            
            # Print what was added to the model (variables and coefficients)
            print(f"Added constraint flow_enforcing_road_{w}_{e}_{t}")
            print(f"  Variables: {expr_vars}")
            print(f"  Coefficients: {expr_coeffs}")

        except KeyError as ke:
            print(f"KeyError: {ke} - This indicates a missing variable for (e={e}, t={t})")

Added constraint flow_enforcing_road_wide_node1node7_1
  Variables: ['flow_timber_node1node7_1', 'flow_timber_node7node1_1', 'C_node1node7_timber_1', 'M_node1node7_timber_1', 'C_node1node7_wide_1', 'M_node1node7_wide_1', 'U_node1node7_1']
  Coefficients: [1, 1, -5, -5, -5, -5, -5]
Added constraint flow_enforcing_road_wide_node1node6_1
  Variables: ['flow_timber_node1node6_1', 'flow_timber_node6node1_1', 'C_node1node6_timber_1', 'M_node1node6_timber_1', 'C_node1node6_wide_1', 'M_node1node6_wide_1', 'U_node1node6_1']
  Coefficients: [1, 1, -5, -5, -5, -5, -5]
Added constraint flow_enforcing_road_wide_node1node10_1
  Variables: ['flow_timber_node1node10_1', 'flow_timber_node10node1_1', 'C_node1node10_timber_1', 'M_node1node10_timber_1', 'C_node1node10_wide_1', 'M_node1node10_wide_1', 'U_node1node10_1']
  Coefficients: [1, 1, -5, -5, -5, -5, -5]
Added constraint flow_enforcing_road_wide_node1node25_1
  Variables: ['flow_timber_node1node25_1', 'flow_timber_node25node1_1', 'C_node1node25_tim

##### verify
should be 5 x 73 constraints

In [872]:
print(len(model.linear_constraints.get_names()))
n_c4 = len(model.linear_constraints.get_names()) - n_c1 - n_c2 - n_c3
print(n_c4)
for i in range(730, model.linear_constraints.get_num()):
    indices = model.linear_constraints.get_rows(i).ind
    coefs = model.linear_constraints.get_rows(i).val
    name = model.linear_constraints.get_names(i)
    rhs   = model.linear_constraints.get_rhs(i)
    sense = model.linear_constraints.get_senses(i)

    vars_in_constraint = [model.variables.get_names(idx) for idx in indices]
    terms = [f"{coef}*{var}" for coef, var in zip(coefs, vars_in_constraint)]
    expression = " + ".join(terms)
    
    print(f"{name or 'unnamed'}: {expression} {sense} {rhs}")

1460
365
Constraint_flow_enforcing_road_wide_node1node7_1: -5.0*C_node1node7_timber_1 + -5.0*C_node1node7_wide_1 + -5.0*M_node1node7_timber_1 + -5.0*M_node1node7_wide_1 + -5.0*U_node1node7_1 + 1.0*flow_timber_node1node7_1 + 1.0*flow_timber_node7node1_1 L 0.0
Constraint_flow_enforcing_road_wide_node1node6_1: -5.0*C_node1node6_timber_1 + -5.0*C_node1node6_wide_1 + -5.0*M_node1node6_timber_1 + -5.0*M_node1node6_wide_1 + -5.0*U_node1node6_1 + 1.0*flow_timber_node1node6_1 + 1.0*flow_timber_node6node1_1 L 0.0
Constraint_flow_enforcing_road_wide_node1node10_1: -5.0*C_node1node10_timber_1 + -5.0*C_node1node10_wide_1 + -5.0*M_node1node10_timber_1 + -5.0*M_node1node10_wide_1 + -5.0*U_node1node10_1 + 1.0*flow_timber_node1node10_1 + 1.0*flow_timber_node10node1_1 L 0.0
Constraint_flow_enforcing_road_wide_node1node25_1: -5.0*C_node1node25_timber_1 + -5.0*C_node1node25_wide_1 + -5.0*M_node1node25_timber_1 + -5.0*M_node1node25_wide_1 + -5.0*U_node1node25_1 + 1.0*flow_timber_node1node25_1 + 1.0*flow_ti

## Solve

In [ ]:
# Solve the model
model.solve()

# Get and print the solution
solution = model.solution
if solution.is_primal_feasible():
    print(f"Objective value: {solution.get_objective_value()}")
    for var_name in model.variables.get_names():
        print(f"{var_name} = {solution.get_values(var_name)}")
else:
    print("No feasible solution found.")

In [ ]:
# Solve
model.solve()
solution = model.solution.get_values()
print(f"Optimal values: {solution}")

## 💾 store the results

## Visualize the result